In [ ]:
!pip install tqdm
!pip install bs4

In [ ]:
!.\venv\Scripts\jupyter labextension install jupyterlab-plotly
!.\venv\Scripts\jupyter labextension install @jupyter-widgets/jupyterlab-manager plotlywidget

In [ ]:
import os
import sys
import glob
import json
from tqdm import tqdm

sys.path.append('src')

import logging
from datetime import datetime, timezone, timedelta
import re
import requests
import queue

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import ephem
import bs4

In [ ]:
def load_navstar_dict():
    # https://www.n2yo.com/satellites/?c=20
    with open('data/gnss_info.html', mode='r') as f:
        table = ''.join(f.readlines())
    NAVSTAR_DICT = {}
    df = []
    soup = bs4.BeautifulSoup(table, 'html.parser')
    keys = []
    for i, row in enumerate(soup.find_all('tr')):
        if i == 0:
            for td in row.find_all('th'):
                keys.append(td.text.strip())
            continue
        navstar = {}
        for key, td in zip(keys, row.find_all('td')):
            navstar[key] = td.text
        if len(navstar) == 0:
            continue
        name = navstar['Name']
        if name.find('NAVSTAR') != 0:
            continue
        NAVSTAR_DICT[name] = navstar
        df.append(navstar)


    # NAVSTAR 82 (USA 343)
    navstar = {
        'Name': 'NAVSTAR 82 (USA 343)',
        'NORAD ID': '55268',
        'Int\'l Code': '',
        'Launch date': '',
        'Period[minuetes]': '',
        'PRN': '28',
    }
    NAVSTAR_DICT['NAVSTAR 82 (USA 343)'] = navstar
    return NAVSTAR_DICT

NAVSTAR_DICT = load_navstar_dict()
with open('data/navstar.json', mode='w') as f:
    json.dump(NAVSTAR_DICT, f, indent=2)

In [ ]:
def calc_xy(phi_deg, lambda_deg, phi0_deg, lambda0_deg):
    """ 緯度経度を平面直角座標に変換する
    - input:
        (phi_deg, lambda_deg): 変換したい緯度・経度[度]（分・秒でなく小数であることに注意）
        (phi0_deg, lambda0_deg): 平面直角座標系原点の緯度・経度[度]（分・秒でなく小数であることに注意）
    - output:
        x: 変換後の平面直角座標[m]
        y: 変換後の平面直角座標[m]
    """
    # 緯度経度・平面直角座標系原点をラジアンに直す
    phi_rad = np.deg2rad(phi_deg)
    lambda_rad = np.deg2rad(lambda_deg)
    phi0_rad = np.deg2rad(phi0_deg)
    lambda0_rad = np.deg2rad(lambda0_deg)

    # 補助関数
    def A_array(n):
        A0 = 1 + (n**2)/4. + (n**4)/64.
        A1 = -     (3./2)*( n - (n**3)/8. - (n**5)/64. ) 
        A2 =     (15./16)*( n**2 - (n**4)/4. )
        A3 = -   (35./48)*( n**3 - (5./16)*(n**5) )
        A4 =   (315./512)*( n**4 )
        A5 = -(693./1280)*( n**5 )
        return np.array([A0, A1, A2, A3, A4, A5])

    def alpha_array(n):
        a0 = np.nan # dummy
        a1 = (1./2)*n - (2./3)*(n**2) + (5./16)*(n**3) + (41./180)*(n**4) - (127./288)*(n**5)
        a2 = (13./48)*(n**2) - (3./5)*(n**3) + (557./1440)*(n**4) + (281./630)*(n**5)
        a3 = (61./240)*(n**3) - (103./140)*(n**4) + (15061./26880)*(n**5)
        a4 = (49561./161280)*(n**4) - (179./168)*(n**5)
        a5 = (34729./80640)*(n**5)
        return np.array([a0, a1, a2, a3, a4, a5])

    # 定数 (a, F: 世界測地系-測地基準系1980（GRS80）楕円体)
    m0 = 0.9999 
    a = 6378137.
    F = 298.257222101

    # (1) n, A_i, alpha_iの計算
    n = 1. / (2*F - 1)
    A_array = A_array(n)
    alpha_array = alpha_array(n)

    # (2), S, Aの計算
    A_ = ( (m0*a)/(1.+n) )*A_array[0] # [m]
    S_ = ( (m0*a)/(1.+n) )*( A_array[0]*phi0_rad + np.dot(A_array[1:], np.sin(2*phi0_rad*np.arange(1,6))) ) # [m]

    # (3) lambda_c, lambda_sの計算
    lambda_c = np.cos(lambda_rad - lambda0_rad)
    lambda_s = np.sin(lambda_rad - lambda0_rad)

    # (4) t, t_の計算
    t = np.sinh( np.arctanh(np.sin(phi_rad)) - ((2*np.sqrt(n)) / (1+n))*np.arctanh(((2*np.sqrt(n)) / (1+n)) * np.sin(phi_rad)) )
    t_ = np.sqrt(1 + t*t)

    # (5) xi', eta'の計算
    xi2  = np.arctan(t / lambda_c) # [rad]
    eta2 = np.arctanh(lambda_s / t_)

    # (6) x, yの計算
    x = A_ * (xi2 + np.sum(np.multiply(alpha_array[1:],
                                       np.multiply(np.sin(2*xi2*np.arange(1,6)),
                                                   np.cosh(2*eta2*np.arange(1,6)))))) - S_ # [m]
    y = A_ * (eta2 + np.sum(np.multiply(alpha_array[1:],
                                        np.multiply(np.cos(2*xi2*np.arange(1,6)),
                                                    np.sinh(2*eta2*np.arange(1,6)))))) # [m]
    # return
    return x, y # [m]

In [ ]:
def get_gnss_list():
    print(f'get_gnss_list start.')
    NAVSTAR_DICT = load_navstar_dict()
    try:
        gnss_resp = requests.get('https://www.celestrak.com/NORAD/elements/gnss.txt')
        lines = gnss_resp.text.splitlines()
        if lines[0].find('NAVSTAR') < 0:
            raise(Exception())
        with open('./data/gnss.txt', mode='w') as f:
            for line in lines:
                f.write(line + '\n')
    except Exception:
        print(f'gnss_read_from cache.')
        with open('./data/gnss.txt', mode='r') as f:
            text = f.read()
        lines = text.splitlines()
    
    gnss_list = {}
    for i in range(0, len(lines), 3):
        satellite_desc = lines[i]
        if satellite_desc.find('NAVSTAR ') >= 0:
            id = NAVSTAR_DICT[satellite_desc.strip()]['PRN']
            id = ('0' + id)[-2:]
            gnss_name = f'G{id}'
            satellite = ephem.readtle(gnss_name, lines[i + 1], lines[i + 2])
            gnss_list[gnss_name] = satellite
    print(f'loaded {len(gnss_list)} Navstars.')
    return gnss_list

def _load_gnss_data(glob_path:str):
    files = glob.glob(glob_path)
    files.sort()
    messages = []
    for file in tqdm(files):
        file = file.replace('\\','/')
        with open(file, mode='r') as f:
            try:
                message = json.load(f)
            except Exception:
                continue
        file_name = file.split('/')[-1]
        message['message_time'] = datetime.strptime(file_name[:14], '%Y%m%d%H%M%S')
        messages.append(message)
    return messages


def load_gnss_data(target_date:str, device_id:str = 'PC-124',):
    """
    ログディレクトリから、GNSSデータの読み込み
    target_date: 対象日付
    device_id: デバイス名
    """

    date_st= datetime.strptime(target_date, '%Y%m%d').strftime('%Y/%m/%d')

    gsv_match = f'data/{device_id}/{date_st}/**/*GSV*.json'
    gsv_messages = _load_gnss_data(gsv_match)

    gga_match = f'data/{device_id}/{date_st}/**/*GGA*.json'
    gga_messages = _load_gnss_data(gga_match)

    gsa_match = f'data/{device_id}/{date_st}/**/*GSA*.json'
    gsa_messages = _load_gnss_data(gsa_match)
    
    return gsv_messages, gga_messages, gsa_messages


def make_keys(gsv_messages):
    # 時間軸の作成
    start_time = gsv_messages[0]['message_time']
    end_time = gsv_messages[-1]['message_time']

    for message in gsv_messages:
        message_time = message['message_time']
        if message_time < start_time:
            start_time = message_time
        if message_time > end_time:
            end_time = message_time

    time_keys = []
    measure_time = start_time
    while measure_time <= end_time:
        time_keys.append(measure_time)
        measure_time = measure_time + timedelta(seconds=1)
    return time_keys


def get_sv_prns(gsv_messages):
    # 衛星一覧の作成
    sv_prns = list(gnss_list.keys())
    for message in gsv_messages:
        sv_prn = 'G' + message['sv_prn']
        if sv_prn not in sv_prns:
            sv_prns.append(sv_prn)
    sv_prns.sort()
    return sv_prns

def satelite_posision_df(gsv_messages):
    satelite_posision_df = pd.DataFrame(gsv_messages)
    satelite_posision_df['sv_prn'] = 'G' + satelite_posision_df['sv_prn']

    start_time = gsv_messages[0]['message_time']
    end_time = gsv_messages[-1]['message_time']
    time_keys = np.arange(start_time, end_time,np.timedelta64(1, 's'), dtype='datetime64')

    sv_prns = list(gnss_list.keys())
    sv_prns.extend(satelite_posision_df['sv_prn'].unique())
    sv_prns = set(sv_prns)

    index_df = pd.DataFrame(np.array(np.meshgrid(time_keys, list(sv_prns))).T.reshape(-1, 2), columns=['message_time', 'sv_prn'])

    satelite_posision_df = pd.merge(index_df, satelite_posision_df, how='outer')
    satelite_posision_df = satelite_posision_df.set_index('message_time')
    return satelite_posision_df

def make_observer_df(gga_messages, gsa_messages):
    
    start_time = min(gga_messages[0]['message_time'], gsa_messages[0]['message_time'])
    end_time = max(gga_messages[-1]['message_time'], gsa_messages[-1]['message_time'])

    for message in gsv_messages:
        message_time = message['message_time']
        if message_time < start_time:
            start_time = message_time
        if message_time > end_time:
            end_time = message_time

    time_keys = []
    measure_time = start_time
    while measure_time <= end_time:
        time_keys.append(measure_time)
        measure_time = measure_time + timedelta(seconds=1)
    
    lats = []
    lons = []
    pdops = []
    hdops = []
    vdops = []

    lats = np.zeros(len(time_keys))
    lats[:] = np.nan

    lons = np.zeros(len(time_keys))
    lons[:] = np.nan

    pdops = np.zeros(len(time_keys))
    pdops[:] = np.nan
    hdops = np.zeros(len(time_keys))
    hdops[:] = np.nan
    vdops = np.zeros(len(time_keys))
    vdops[:] = np.nan


    for message in gga_messages:
        index = (message['message_time'] - time_keys[0]).seconds
        lats[index] = message['lat']
        lons[index] = message['lon']

    for message in gsa_messages:
        index = (message['message_time'] - time_keys[0]).seconds
        pdops[index] = message['pdop']
        hdops[index] = message['hdop']
        vdops[index] = message['vdop']

    observer_df = pd.DataFrame(
        {
            'lat': lats,
            'lon': lons,
            'pdop': pdops,
            'hdop': hdops,
            'vdop': vdops,
        }, 
        index=time_keys
    )
    return observer_df


In [ ]:
def plot_observer_df(observer_df, start_time_st: str = '', range_sec = 0):
    rangeslider = True
    if len(start_time_st) > 0:
        view_start = datetime.strptime(start_time_st, '%Y%m%d%H%M%S')
        view_end = view_start + timedelta(seconds=range_sec)
        observer_df = observer_df[(observer_df.index >= view_start) & (observer_df.index < view_end)].dropna(how='any')
        rangeslider = False
    
    x_max, x_min = observer_df['x'].describe()[['max', 'min']]
    y_max, y_min = observer_df['y'].describe()[['max', 'min']]
    
    print(f'x_rng: {x_max - x_min:.1f}')
    print(f'y_rng: {y_max - y_min:.1f}')
    
    display(observer_df.describe())
    time_keys = np.array(observer_df.index)
    lats = observer_df['lat']
    lons = observer_df['lon']

    x = observer_df['x'] - observer_df['x'].mean()
    y = observer_df['y'] - observer_df['y'].mean()
    
    pdops = observer_df['pdop']
    hdops = observer_df['hdop']
    vdops = observer_df['vdop']
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=observer_df.index,
            y=pdops,
            mode='lines',
            name='pdop',
            yaxis='y1',
            opacity=0.5,
        ),
    )
    fig.add_trace(
        go.Scatter(
            x=observer_df.index,
            y=hdops,
            mode='lines',
            name='hdop',
            yaxis='y1',
            opacity=0.5,
        ),
    )
    fig.add_trace(
        go.Scatter(
            x=observer_df.index,
            y=vdops,
            mode='lines',
            name='vdop',
            yaxis='y1',
            opacity=0.5,
        ),
    )
    """
    fig.add_trace(
        go.Scatter(
            x=observer_df.index,
            y=lats,
            mode='lines',
            name='latitude',
            yaxis='y3',
        ),
    )
    fig.add_trace(
        go.Scatter(
            x=observer_df.index,
            y=lons,
            mode='lines',
            name='longitude',
            yaxis='y2',
        ),
    )
    """
    fig.add_trace(
        go.Scatter(
            x=observer_df.index,
            y=x,
            mode='lines',
            name='EW',
            yaxis='y2',
        ),
    )
    fig.add_trace(
        go.Scatter(
            x=observer_df.index,
            y=y,
            mode='lines',
            name='NS',
            yaxis='y2',
        ),
    )

    fig.update_layout(
        dict(
            title='OBSERVER POSITION',
            hovermode='x',
            height=600,
            width=800,
            xaxis=dict(
                rangeslider={"visible":rangeslider},
                domain=[0.2, 1.0]
            ),
            yaxis=dict(
                # title='hdop',
                range=(0,10.0),
            ),
            yaxis2=dict(
                overlaying='y',
                side="right",
                # title='longitude',
                range=(-25,25)
            ),
            
            #yaxis3=dict(
            #    anchor="free",
            #    position=0.1,
            #    overlaying='y',
            #    # title='latitude',
            #),
        )
    )
    return fig

def plot_map(observer_df, dop_thresh: int = 3.0):
    fig = go.Figure()

    lats = observer_df['lat']
    lons = observer_df['lon']
    pdops = observer_df['pdop']
    
    lons_good = lons.values.copy()
    lats_good = lats.values.copy()
    lons_good[pdops >= dop_thresh] = np.nan
    lats_good[pdops >= dop_thresh] = np.nan

    lons_bad = lons.values.copy()
    lats_bad = lats.values.copy()
    lons_bad[pdops < dop_thresh] = np.nan
    lats_bad[pdops < dop_thresh] = np.nan

    fig.add_trace(
        go.Scattermapbox(
            mode = "lines",
            lon = lons_good,
            lat = lats_good,
            text = observer_df.index,
        )
    )
    fig.add_trace(
        go.Scattermapbox(
            mode = "lines",
            lon = lons_bad,
            lat = lats_bad,
            text = observer_df.index,
        )
    )
    fig.update_layout(
        height=600,
        width=800,
        autosize=True,
        hovermode='closest',
        mapbox=dict(
            style='open-street-map',
            bearing=0,
            pitch=0,
            zoom=17,
            center=dict(
                lat=np.nanmedian(lats),
                lon=np.nanmedian(lons),
            ),
        ),
    )
    return fig


def plot_srn(gsv_df, ):
    datas = []
    fig = go.Figure()
    srn_df = gsv_df.dropna(how='any')[['sv_prn', 'srn']].pivot_table(values=['srn'], index=['message_time'], columns=['sv_prn'], aggfunc='mean').fillna(0).resample('15S').mean()
    for sv_prn in srn_df.columns:
        srns = srn_df[sv_prn]
        fig.add_trace(
            go.Scatter(
                x=srn_df.index,
                y=srns,
                mode='lines',
                name=sv_prn[1],
            )
        )
    fig.update_layout(
        dict(
            title='GNSS SRN',
            hovermode='x',
            height=600,
            width=800,
            xaxis=dict(
                rangeslider={"visible":True},
            ),
            yaxis=dict(
                title='SRN(dB)',
                range=(0, 100)
            ),
        )
    )
    return fig

def plot_srn_summary(gsv_df, observer_df):
    
    pdops = observer_df['pdop']
    srn_df = gsv_df.dropna(how='any')[['sv_prn', 'srn']].pivot_table(values=['srn'], index=['message_time'], columns=['sv_prn'], aggfunc='mean').fillna(0).resample('15S').mean()

    sum_column = srn_df.sum(axis=1)
    cnt_column = srn_df[srn_df > 0].count(axis=1)
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=cnt_column.index,
            y=cnt_column.values,
            mode='lines',
            name='count',
            yaxis='y1',
        ),
    )
    fig.add_trace(
        go.Scatter(
            x=sum_column[cnt_column > 0].index,
            y=sum_column[cnt_column > 0].values/cnt_column[cnt_column > 0].values,
            mode='lines',
            name='srn average',
            yaxis='y2',
        ),
    )
    fig.add_trace(
        go.Scatter(
            x=observer_df.index,
            y=pdops,
            mode='lines',
            name='pdop',
            yaxis='y3',
            opacity=0.5,
        ),
    )
    fig.update_layout(
        dict(
            title='SRN SUMMARY',
            hovermode='x',
            height=600,
            width=800,
            xaxis=dict(
                rangeslider={"visible":True},
                domain=[0.1, 1.0]
            ),
            yaxis=dict(
                # title='latitude',
                range=(0, 16),
            ),
            yaxis2=dict(
                overlaying='y',
                side="right",
                range=(0, 100),
            ),
            yaxis3=dict(
                overlaying='y',
                range=(0, 10),
                position=0.05,
            ),
        )
    )
    return fig



def make_srn_bar_chart(observer_df, gsv_df, start_time_st, range_sec):
    
    view_start = datetime.strptime(start_time_st, '%Y%m%d%H%M%S')
    view_end = view_start + timedelta(seconds=range_sec)
    
    srn_max = gsv_df['srn'].max()
    srn_max = gsv_df.dropna(how='any')[['sv_prn', 'srn']].groupby('sv_prn').max()
    srn_max = srn_max.rename(columns={'srn': 'srn_max'}).reset_index()

    df = gsv_df[(gsv_df.index >= view_start) & (gsv_df.index < view_end)][['sv_prn', 'srn',]].dropna()
    df = df.groupby('sv_prn').mean().reset_index()
    df = pd.merge(df, srn_max, how='outer').set_index('sv_prn')
    df = df.sort_values('srn', ascending=False)
    df = df[:15]
    
    display(df[df['srn'] > 0]['srn'].describe())
    
    fig = go.Figure()
    fig.add_traces(
        data=[
            # 電波強度
            go.Bar(
                x=df.index,
                y=df['srn'],
                xaxis='x1',
                yaxis='y1',
                marker=dict(
                    color='blue',
                    opacity=0.5,
                )
            ),
        ],
    )
    fig.update_layout(
        dict(
            # title='SRN Strength',
            hovermode='x',
            height=600,
            width=600,
            xaxis=dict(
                dtick=1
            ),
            yaxis=dict(
                # title='SRN(dB)',
                range=(0, 40)
            ),
        )
    )

    return fig



def make_satelite_graph(observer_df, gsv_df, start_time_st, range_sec):
    
    view_start = datetime.strptime(start_time_st, '%Y%m%d%H%M%S')
    view_end = view_start + timedelta(seconds=range_sec)
    observer_data = observer_df.iloc[observer_df.index.get_loc(view_start, method='nearest')]
    center_time = view_start + timedelta(seconds=range_sec / 2)
    observer_data = observer_df[(observer_df.index >= view_start) & (observer_df.index < view_end)].mean()
    
    # 中間時間で観測点を作成
    observer = ephem.Observer()
    observer.lat = str(observer_data['lat'])
    observer.lon = str(observer_data['lon'])
    observer.elevation = 0.0
    observer.date = view_start + timedelta(seconds=range_sec / 2) - timedelta(hours=9)
    
    # 衛星別の電波強度・位置情報を計算
    gnss_names = []
    el_degree_n = []
    az_degree_n = []
    for gnss_name, satellite in gnss_list.items():
        gnss_names.append(gnss_name)
        satellite.compute(observer)
        el_degree = satellite.alt * 180 / np.pi
        el_degree_n.append(el_degree)
        az_degree = satellite.az * 180 / np.pi
        az_degree_n.append(az_degree)
    
    norad_datas = pd.DataFrame([el_degree_n, az_degree_n]).transpose()
    norad_datas.index = gnss_names
    norad_datas = norad_datas.set_axis(['el_degree_n', 'az_degree_n'], axis='columns')
    
    srn_max = gsv_df['srn'].max()
    srn_max = gsv_df.dropna(how='any')[['sv_prn', 'srn']].groupby('sv_prn').max()
    srn_max = srn_max.rename(columns={'srn': 'srn_max'}).reset_index()

    df = gsv_df[(gsv_df.index >= view_start) & (gsv_df.index < view_end)][['sv_prn', 'srn', 'el_degree', 'az_degree']].dropna()
    df = df.groupby('sv_prn').mean().reset_index()
    df = pd.merge(df, srn_max, how='outer').set_index('sv_prn')
    df = df.sort_index()
    df = df[(df['el_degree'] != 0.0) & (df['az_degree'] != 0.0)]
    
    df_position = df.dropna(how='any')
    df = df.fillna(0)
    
    # 軌道を計算
    orbits = {}
    for gnss_name, satellite in gnss_list.items():
        el_degrees = []
        az_degrees = []
        rising = True
        setting = True
        interval=15
        hours = 6
        for i in range(0, hours*60, interval):
            observer.date = datetime.utcnow() - timedelta(minutes=i)
            satellite.compute(observer)
            if rising and (satellite.alt >= 0):
                el_degrees.insert(0, satellite.alt * 180 / np.pi)
                az_degrees.insert(0, satellite.az * 180 / np.pi)
            else:
                rising = False

            observer.date = datetime.utcnow() + timedelta(minutes=i)
            satellite.compute(observer)
            if setting or (satellite.alt >= 0):
                el_degrees.append(satellite.alt * 180 / np.pi)
                az_degrees.append(satellite.az * 180 / np.pi)
            else:
                setting = False

            if (rising == False) and (setting == False):
                break

        orbits[gnss_name] = {
            'el_degrees': el_degrees,
            'az_degrees': az_degrees,
        }

    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'polar'}, {'type': 'xy'}]])
    
    # レーダーチャートの作成
    layout = go.Layout(
        title=f"{name} : {center_time.strftime('%Y/%m/%d %H:%M:%S')} " + \
              f"@( {float(observer_data['lat']):.6f}, {float(observer_data['lon']):.6f} )",
        polar=dict(
            angularaxis=dict(
                direction='clockwise',
            ),
            radialaxis=dict(
                range=[0, 90],
                showticklabels=False,
            )
        ),
        autosize=True,
        xaxis1=dict(),
        yaxis1=dict(range=[0, 60], dtick=10),
        showlegend=False,
    )

    fig.add_traces(
        # 衛星位置
        data=[
            # 観測位置
            go.Scatterpolar(
                theta=df_position['az_degree'].astype(np.int64),
                r=90.0 - df_position['el_degree'].astype(np.int64),
                mode='markers+text',
                marker=dict(
                    color='blue',
                    opacity=0.5,
                    size=np.array(df_position['srn'])
                ),
                text=df_position.index,
                textposition='top center',
            ),
            # NORAD位置
            go.Scatterpolar(
                theta=norad_datas['az_degree_n'].astype(np.int64),
                r=90.0 - norad_datas['el_degree_n'].astype(np.int64),
                mode='markers',
                marker=dict(
                    color='red',
                    opacity=0.5,
                    symbol=4,
                ),
                text=norad_datas.index,
                textposition='top center',
            ),
        ],
        rows=[1, 1],
        cols=[1, 1],
    )
    for gnss_name, orbit in orbits.items():
        fig.add_trace(
            # 衛星軌跡
            trace=go.Scatterpolar(
                theta=np.array(orbit['az_degrees']),
                r=90.0 - np.array(orbit['el_degrees']),
                mode='lines',
                line=dict(
                    color='red',
                ),
                text=gnss_name,
                opacity=0.1,
                hoverinfo='skip',
            ),
            row=1,
            col=1,
        )
    fig.add_traces(
        data=[
            # 電波強度
            go.Bar(
                x=df.index,
                y=df['srn'],
                xaxis='x1',
                yaxis='y1',
                marker=dict(
                    color='blue',
                    opacity=0.5,
                )
            ),
            # 電波強度（最大）
            go.Bar(
                x=df.index,
                y=df['srn_max'],
                xaxis='x1',
                yaxis='y1',
                marker=dict(
                    color='red',
                    opacity=0.5,
                )
            ),
        ],
        rows=[1, 1],
        cols=[2, 2],
    )

    fig.update_layout(
        layout,
    )
    return fig

In [ ]:
gnss_list = get_gnss_list()

In [ ]:
target_date = '20231127'

In [ ]:
gsv_messages, gga_messages, gsa_messages = load_gnss_data(target_date)

# sv_prns = get_sv_prns(gsv_messages)
gsv_df = satelite_posision_df(gsv_messages)
observer_df = make_observer_df(gga_messages, gsa_messages)

In [ ]:
gsv_df.reset_index().to_csv(f'data/gsv_df_{target_date}.csv', index=False)
observer_df.reset_index().to_csv(f'data/observer_df_{target_date}.csv', index=False)

In [ ]:
gsv_df = pd.read_csv(f'data/gsv_df_{target_date}.csv')
observer_df = pd.read_csv(f'data/observer_df_{target_date}.csv')
gsv_df = gsv_df.set_index('message_time')
observer_df = observer_df.set_index('message_time')
gsv_df.index = pd.to_datetime(gsv_df.index)
observer_df.index = pd.to_datetime(observer_df.index)

In [ ]:
display(gsv_df)
display(observer_df)

In [ ]:
max_srn = gsv_df[['sv_prn', 'srn']].groupby('sv_prn').max().reset_index()
min_srn = gsv_df[['sv_prn', 'srn']].groupby('sv_prn').min().reset_index()
max_srn.columns=['sv_prn', 'srn_max']
max_srn.columns=['sv_prn', 'srn_min']

pd.merge(max_srn, min_srn, how='outer')

In [ ]:
# 位置情報・精度のグラフ描画
fig = plot_observer_df(observer_df)
fig.show()
fig.write_html(f'data/dop_{target_date}.html')

In [ ]:
# 地図上へ描画
fig = plot_map(observer_df, dop_thresh=3.0)
fig.show()
fig.write_html(f'data/map_{target_date}.html')

In [ ]:
fig = plot_srn(gsv_df)
fig.show()
fig.write_html(f'data/srn_{target_date}.html')

In [ ]:
fig = plot_srn_summary(gsv_df, observer_df)
fig.show()
fig.write_html(f'data/srn_summary_{target_date}.html')

In [ ]:
# center_time_st = '2021/11/18 14:07:55'
# name = '検査場'

#center_time_st = '2021/11/18 13:44:38'
#name = 'C棟'

#center_time_st = '2021/11/18 14:04:09'
#name = '組立ライン'

#center_time_st = '2021/11/18 14:57:47'
#name = 'ショベル実験エリア'


# center_time_st = '20211118145747'
# name = 'ショベル実験エリア'

#center_time_st = '2022/04/08 11:50:00'
#name = '駐車場'

#center_time_st = '2022/04/08 13:00:00'
#name = '杉林'


# start_time = '20231127103000'
# name = '屋外：ダッシュボード内（樹脂）'

# start_time = '20231127104501'
# name = '屋外：ダッシュボード内（板金）'

# start_time = '20231127110002'
# name = '屋外：足元（板金）'

# start_time = '20231127112500'
# name = '屋外：後部に設置'

# start_time = '20231127130400'
# name = '屋内：ダッシュボード内（樹脂）'

# start_time = '20231127132000'
# name = '屋内：ダッシュボード内（板金）'

# start_time = '20231127133700'
# name = '屋内：足元（板金）'

start_time = '20231127114900'
name = '屋内：後部に設置'


range_sec = 10*60

fig = make_srn_bar_chart(observer_df, gsv_df, start_time, range_sec)
fig.show()

fig = make_satelite_graph(observer_df, gsv_df, start_time, range_sec)
fig.write_html(f'data/{name}_{start_time}.html')
fig.update_layout(
    width=800,
    height=500,
)
fig.show()

fig = plot_observer_df(observer_df,  start_time, range_sec)
fig.show()
